# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [33]:
from collections import Counter
import math

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

import problem_unittests as tests
import helper

In [2]:
def get_local_device() -> torch.device:
    available_device = "cuda" if torch.cuda.is_available() else "cpu"
    return torch.device(available_device)

device = get_local_device()
print(device)

cuda


In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [2]:
view_line_range = (100100, 100110)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
word_count_line = [count for count in word_count_line if count != 0]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))
print('Median number of words in each line: {}'.format(np.median(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 11.088582100483375
Median number of words in each line: 8.0

The lines 100100 to 100110:
elaine: well, it's funny. i mean, you have a big head. or is it 'cause of your neck? 

jerry: no, i think the head does most of the stretching. 

george: regardless. i had to walk around for the rest of the party in some cheap metlife windbreaker. now, it is payback time. 

elaine: i really think it's the size of your neck. 

george: it's my head! 



---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [3]:
def get_unique_words_sorted_by_frequency(text: list) -> list:
    word_counts = Counter(text)
    sorted_counts = word_counts.most_common()
    sorted_words = dict(sorted_counts).keys()
    return sorted_words


def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    vocabulary = get_unique_words_sorted_by_frequency(text)
    int_to_vocab = dict(enumerate(vocabulary))
    vocab_to_int = dict(zip(int_to_vocab.values(), int_to_vocab.keys()))
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    punctuation_tokens = {'.': "<PERIOD>",
                          ',': "<COMMA>",
                          '"': "<QUOTE>",
                          ';': "<SEMICOLON>",
                          '!': "<EXCLAMATION>",
                          '?': "<QUESTION>",
                          '(': "<LPARANTHESES>",
                          ')': "<RPARANTHESES>",
                          '-': "<DASH>",
                          '\n': "<RETURN>"}
    return punctuation_tokens

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to look at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [3]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [4]:
def get_next_data_sequence(words: list, sequence_length: int) -> (list, int):
    word_count = len(words)
    scan_index = 0
    last_scan_index = word_count - sequence_length -1
    
    while scan_index <= last_scan_index:
        features = words[scan_index:scan_index+sequence_length]
        target = words[scan_index+sequence_length]
        scan_index += 1
        yield features, target


def build_data_tensors(words: list, sequence_length: int) -> (torch.Tensor, torch.Tensor):
    features_list, targets_list = [], []
    for features, targets in get_next_data_sequence(words, sequence_length):
        features_list.append(features)
        targets_list.append(targets)
        
    features_tensor = torch.LongTensor(features_list)
    targets_tensor = torch.LongTensor(targets_list)
    return features_tensor, targets_tensor


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    features, targets = build_data_tensors(words, sequence_length)
    data = TensorDataset(features, targets)
    loader = DataLoader(dataset=data, batch_size=batch_size, shuffle=True)
    return loader

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own

data = int_text[:12]
print(data)
gen = get_next_data_sequence(data, 10)
print(next(gen))
print(next(gen))
build_data_tensors(data, 10)

[24, 22, 47, 1, 1, 1, 17, 47, 22, 82, 20, 6]
([24, 22, 47, 1, 1, 1, 17, 47, 22, 82], 20)
([22, 47, 1, 1, 1, 17, 47, 22, 82, 20], 6)


(tensor([[24, 22, 47,  1,  1,  1, 17, 47, 22, 82],
         [22, 47,  1,  1,  1, 17, 47, 22, 82, 20]]),
 tensor([20,  6]))

### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [5]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[43, 44, 45, 46, 47],
        [33, 34, 35, 36, 37],
        [31, 32, 33, 34, 35],
        [ 2,  3,  4,  5,  6],
        [17, 18, 19, 20, 21],
        [13, 14, 15, 16, 17],
        [28, 29, 30, 31, 32],
        [ 1,  2,  3,  4,  5],
        [25, 26, 27, 28, 29],
        [18, 19, 20, 21, 22]])

torch.Size([10])
tensor([48, 38, 36,  7, 22, 18, 33,  6, 30, 23])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [6]:
class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super().__init__()
        self.embed = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=n_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(in_features=hidden_dim, out_features=output_size)
        self.fc_dropout = nn.Dropout(p=dropout)
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        long_int_input = nn_input.type(torch.LongTensor).to(self.device)
        embedded_input = self.embed(long_int_input)
        lstm_outputs, new_hidden = self.lstm(embedded_input, hidden)
        last_lstm_output = lstm_outputs[:,-1,:]
        last_lstm_output = self.fc_dropout(last_lstm_output)
        decoded_output = self.fc(last_lstm_output)
        
        return decoded_output, new_hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        state_size = (self.lstm.num_layers, batch_size, self.lstm.hidden_size)
        h = torch.zeros(size=state_size, device=self.device)
        c = torch.zeros(size=state_size, device=self.device)
        hidden_state = (h,c)
        return hidden_state

In [7]:
class PositionalEncoding(nn.Module):
    def __init__(self, dimension: int, dropout: int=0.1, max_length: int=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        positional_values = self.__generate_position_values(dimension, max_length)
        self.register_buffer("positional_values", positional_values)
        
    @staticmethod
    def __generate_position_values(dimension: int, max_length: int):
        values = torch.zeros(max_length, dimension)
        positions = torch.arange(0, max_length, dtype=torch.float)
        positions = positions.unsqueeze(1)
        
        scaling_steps = torch.arange(0, dimension, 2).float()
        scaling = torch.exp(scaling_steps * (-math.log(10000.0)/dimension))
        
        values[:, 0::2] = torch.sin(positions * scaling)
        values[:, 1::2] = torch.cos(positions * scaling)
        values = values.unsqueeze(0).transpose(0, 1)
        
        return values
    
    def forward(self, x):
        x = x + self.positional_values[:x.size(0), :]
        return self.dropout(x)

In [21]:
class TransformerScriptGenerator(nn.Module):
    def __init__(self,
                 vocabulary_size: int,
                 embedding_dim: int,
                 attention_head_count: int,
                 encoder_fc_dim: int,
                 encoder_layer_count: int,
                 dropout: int=0.5) -> None:
        
        super().__init__()
        
        self.input_mask = None
        self.embedding = nn.Embedding(vocabulary_size, embedding_dim)
        self.embedding_scale = math.sqrt(embedding_dim)
        self.positional_encoder = PositionalEncoding(embedding_dim, dropout)
        encoder_layers = nn.TransformerEncoderLayer(embedding_dim, attention_head_count, encoder_fc_dim, dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, encoder_layer_count)
        self.embedding_dim = embedding_dim
        self.decoder = nn.Linear(embedding_dim, vocabulary_size)
        
        self.__init_weights()
        
    def __init_weights(self) -> None:
        value_range = 0.1
        self.embedding.weight.data.uniform_(-value_range, value_range)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-value_range, value_range)
        
    @staticmethod
    def __generate_input_mask(mask_size: int) -> torch.Tensor():
        mask = torch.ones(mask_size, mask_size, dtype=bool)
        mask = torch.triu(mask).t().float()
        mask = mask.masked_fill(mask == 0, float('-inf'))
        mask = mask.masked_fill(mask == 1, 0.0)
        return mask
    
    @staticmethod
    def __get_output_for_last_word(full_output: torch.Tensor) -> torch.Tensor:
        return full_output[:,-1,:]
    
    def forward(self, input: torch.Tensor) -> torch.Tensor:
        mask_size = input.shape[0]
        if self.input_mask is None or self.input_mask.size(0) != mask_size:
            self.input_mask = self.__generate_input_mask(mask_size).to(device)
            
        input = self.embedding(input) * self.embedding_scale
        input = self.positional_encoder(input)
        output = self.transformer_encoder(input, self.input_mask)
        output = self.__get_output_for_last_word(output)
        output = self.decoder(output)
        return output

### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [22]:
def truncate_hidden_backprop(hidden_state: (torch.Tensor, torch.Tensor)) -> (torch.Tensor, torch.Tensor):
    new_h = hidden_state[0].clone().detach()
    new_c = hidden_state[0].clone().detach()
    return (new_h, new_c)


def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param rnn: The PyTorch Module that holds the neural network
    :param optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    hidden = truncate_hidden_backprop(hidden)
    optimizer.zero_grad()
    output, new_hidden = rnn.forward(inp.to(device), hidden)
    loss = criterion(output, target.to(device))
    loss.backward()
    nn.utils.clip_grad_norm_(rnn.parameters(), 4.0)
    optimizer.step()
    
    return loss.item(), new_hidden

In [23]:
def transformer_training_step(model: TransformerScriptGenerator, 
                              optimizer: nn.Module, 
                              criterion: nn.Module,
                              input: torch.Tensor, 
                              target: torch.Tensor) -> torch.Tensor:
    optimizer.zero_grad()
    input_batch_second = input.to(device).t()
    output = model(input_batch_second)
    loss = criterion(output, target.to(device))
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), 2.0)
    optimizer.step()
    return loss

In [11]:
model = TransformerScriptGenerator(1000, 128, 4, 1024, 5).to(device)
print(sample_x)
model(sample_x.t().to(device))

tensor([[43, 44, 45, 46, 47],
        [33, 34, 35, 36, 37],
        [31, 32, 33, 34, 35],
        [ 2,  3,  4,  5,  6],
        [17, 18, 19, 20, 21],
        [13, 14, 15, 16, 17],
        [28, 29, 30, 31, 32],
        [ 1,  2,  3,  4,  5],
        [25, 26, 27, 28, 29],
        [18, 19, 20, 21, 22]])
torch.Size([5, 10, 128])


tensor([[[ 0.0479,  0.6834, -0.1532,  ...,  0.6726, -1.2294, -0.3489],
         [-0.3361,  1.2168,  0.3205,  ...,  1.6006,  0.6945, -0.6341],
         [-0.2127,  1.2319, -0.2411,  ..., -0.5731,  0.7166,  0.5457],
         ...,
         [-0.2693, -0.3433,  0.1277,  ...,  0.0927, -0.3072,  0.4069],
         [-0.2754,  0.5777, -0.3360,  ...,  0.2872, -0.5546,  0.6101],
         [ 0.6238,  0.7724, -0.1810,  ..., -0.7811, -0.8033, -0.3356]],

        [[-0.2991,  0.3357,  0.7227,  ..., -0.1378, -0.9689,  0.2474],
         [-1.1027,  0.5761,  0.7901,  ...,  0.9040, -0.3082, -0.0867],
         [-0.1757,  0.7560,  0.4010,  ...,  0.6364, -0.4835,  0.2113],
         ...,
         [ 0.3324, -0.3580,  0.1099,  ...,  0.6830, -0.9911, -0.2817],
         [ 1.3794, -0.0959,  0.1834,  ...,  0.4699,  0.1664, -0.0263],
         [-0.5124,  0.9073,  1.0184,  ...,  0.0670,  0.1618, -0.0799]],

        [[-0.6468,  0.0783,  0.2492,  ...,  0.0034, -0.8417, -1.4943],
         [ 0.8239, -0.8418,  0.6622,  ...,  0

## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [12]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

In [40]:
def should_skip_batch(input_batch: torch.Tensor, batch_size: int) -> bool:
    if input_batch.shape[1] != batch_size:
        print("Invalid batch size; skipping.")
        return True
    return False


def train_transformer(model: TransformerScriptGenerator, 
                      optimizer: nn.Module, 
                      criterion: nn.Module,
                      lr_adjuster: nn.Module,
                      epoch_count: int,
                      train_loader: DataLoader,
                      batch_size: int) -> None:
    
    model.train()
    print(f"Training for {epoch_count} epochs...")
    
    for epoch in range(epoch_count):
        print(f"Epoch {epoch}")
        total_batches = 0
        total_loss = 0
        
        for inputs, targets in train_loader:
            inputs.t_()
            if should_skip_batch(inputs, batch_size):
                continue
            total_batches += 1
            total_loss += transformer_training_step(model, optimizer, criterion, inputs, targets)
            if total_batches%500 == 0:
                print(f"+ Average training loss: {total_loss / total_batches} for {total_batches} batches")
         
        lr_adjuster.step()
        print(f"+ Average training loss: {total_loss / total_batches}")
    

In [44]:
model = TransformerScriptGenerator(vocabulary_size=len(set(int_text))+1, # the extra word is the padding token
                                   embedding_dim=1024, 
                                   attention_head_count=4, 
                                   encoder_fc_dim=512, 
                                   encoder_layer_count=3,
                                   dropout=0.2).to(device)

sequence_length = 30
batch_size = 300
train_loader = batch_data(int_text, sequence_length, batch_size)

epoch_count = 1
learning_rate = 0.001#5.0
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)#torch.optim.SGD(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

In [45]:
train_transformer(model, optimizer, criterion, scheduler, epoch_count, train_loader, batch_size)

Training for 1 epochs...
Epoch 0
+ Average training loss: 6.125511169433594 for 500 batches
+ Average training loss: 5.9640631675720215 for 1000 batches
+ Average training loss: 5.895380020141602 for 1500 batches
+ Average training loss: 5.8594441413879395 for 2000 batches


KeyboardInterrupt: 

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of unique tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [156]:
# Data params
# Sequence Length
sequence_length = 10
# Batch Size
batch_size = 300

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [157]:
# Training parameters
# Number of Epochs
num_epochs = 12
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(set(int_text))+1 # the extra word is the padding token
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 1024
# Hidden Dimension
hidden_dim = 896
# Number of RNN Layers
n_layers = 3

# Show stats for every n number of batches
show_every_n_batches = 500

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [161]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5).to(device)

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 12 epoch(s)...
torch.Size([300, 21388])
torch.Size([300])
torch.Size([300, 21388])
torch.Size([300])
torch.Size([300, 21388])
torch.Size([300])
torch.Size([300, 21388])
torch.Size([300])
torch.Size([300, 21388])
torch.Size([300])
torch.Size([300, 21388])
torch.Size([300])
torch.Size([300, 21388])
torch.Size([300])
torch.Size([300, 21388])
torch.Size([300])
torch.Size([300, 21388])
torch.Size([300])
torch.Size([300, 21388])
torch.Size([300])
torch.Size([300, 21388])
torch.Size([300])
torch.Size([300, 21388])
torch.Size([300])
torch.Size([300, 21388])
torch.Size([300])
torch.Size([300, 21388])
torch.Size([300])
torch.Size([300, 21388])
torch.Size([300])
torch.Size([300, 21388])
torch.Size([300])
torch.Size([300, 21388])
torch.Size([300])
torch.Size([300, 21388])
torch.Size([300])
torch.Size([300, 21388])
torch.Size([300])
torch.Size([300, 21388])
torch.Size([300])
torch.Size([300, 21388])
torch.Size([300])
torch.Size([300, 21388])
torch.Size([300])
torch.Size([300, 21388])
t

KeyboardInterrupt: 

### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:** 
- With the first training session, I wanted to check if the training process will complete  without errors, so I chose hyperparameter values that would have low computational cost. The exact values were chosen arbitrarily, mainly based on previous projects:

sequence len.:  10\
batch size:     50\
epoch count:     1\
learning rate:  0.001\
embedding dim.: 100\
hidden dim.:    128\
LSTM layers:    2
- The network went form the initial loss of 6.00 down to about 4.50, so the training algorithm seemed to be working as intended. The hyperparameters obviously required tuning to reach the goal of 3.5 loss, though.
- First, I increased the step count to 100, hoping that this will enable the network to "see" more relationships between words in these wider temporal windows. Loss was around 4.55 after 1-st epoch and 4.25 after 4-th. So there wasn't actually much of an improvement. So I took a different approach: I calculated the mean and median for non-empty words in a line, and got 8 and 11, respectively. Based on this finding I tested sequence lengths of 8, 10 and 12, during short, 1-epoch sessions. It turned out that the initial value of 10 was producing the best results.
- Then, I tested several batch sizes (24, 50, 100, 200, 300, 400). I was hoping that smaller batches would result in more frequent weight optimization steps and faster convergence. However, the effect was opposite: the additional computational cost was slowing the process heavily and it took way too long. Ultimately, I noticed I can get away with a batch size of 300 without incurring any significant increase of training loss while speeding up training a lot.
- I did some testing of several learning rate values but generally got bad results each time, so finally I settled with the standard (for Adam optimizer) 0.001 LR.
- From the earlier lessons and other sources, I knew that 2 or 3 LSTM layers are usually enough for these types of problems. I compared two models, having the same number of units, but distributed to 2 or 3 layers. The difference in the final loss value was small, but the 3-layer architecture did seem to have slightly better accuracy.
- The two parameters that turned out to have the greatest impact on training loss, were the total number of LSTM units and the embedding dimension. This makes sense – the larger the embedding vector, the more input features are visible to the network. It can then learn more relationships and patterns. And the more units, the more “capacity” for learning it has. I tested embedding values from 100 to 2048 and unit counts of up to 3072. Best results were achieved for models having a large embedding dim AND many LSTM units, but the training times were very long. Other combinations (e.g. small embedding, many units) were not so successful, but faster. Embedding dims of around 512 with around 3 times as many total LSTM units proved to provide a good compromise between accuracy and training time. For the final run, I settled for embedding dim of 1024, 896 units/layer with 3 layers (so 2688 units in total). After 12 epochs, this model reached a final loss of 3.41. 

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [31]:
def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        current_seq = torch.LongTensor(current_seq).to(device)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data.cpu()
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        current_seq = current_seq.cpu() # move to cpu
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [22]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'kramer' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

kramer:)

kramer: hey, what are you doing?

jerry: oh, i'm sure it's a very nice gesture.

jerry: i don't know if i can tell you that i was in the mood and you have to be in a coma.(he starts backing up to the door.) oh my god.

elaine:(to elaine) i thought you knew you were going to be the same person i was wearing.

george: what is that?

elaine:(to george) oh, you think i'm a little too sensitive.

jerry: oh, i can't get a little.

jerry: oh, well, it's not the only way i could get out of here..(he leaves)

george: i can't believe it, jerry- i don't know what you do.

jerry: oh, well, it's a little unusual.

elaine: oh, well, i don't know, what about that. i mean, i don't want to get a little...

elaine:(to the phone) hello? what are we doing here?

jerry: i was thinking. i can't do it.

george: you don't understand, but you know, i was just trying to go.

jerry: what?

george: i got a date.

jerry: well, i think i can have it.

jerry:(to kramer) what are you talking about? you got 

In [38]:
def generate_with_transformer(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        current_seq = torch.LongTensor(current_seq).to(device)
                
        # get the output of the rnn
        output = rnn(current_seq)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data.cpu()
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        current_seq = current_seq.cpu() # move to cpu
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

In [46]:
gen_length = 400 # modify the length to your preference
prime_word = 'kramer' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate_with_transformer(model, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

kramer:??


. you..


,
..
...

.
..
,

,

you

.

.
you
,


.,?,.
you

,
..
?
.
?,


,
...??
?
,.?


,


? you.?. you,
..
?., you
,,..
you,


.
.

,

you
.
you
,,.
you
,,.?
?




you..,



,

you
,. you,

. you. you
,. you,
..?,
,.,,

. you,
,
.?
,
,....
..?,?
.

,,?.
.
..
.
,?. you







.

you

.., you
..

..?.
..
.,
..,
,.



?,.
,

,?.

..
,.,.,
you
.

..
you

, you
,,?,
,. you
.
....,,, you., you.

you
.
.




,
,


,,
?.
,.,.
,

,,.
. you
,



?



.

,.







?

.





#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [19]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.